**Authors:**  <br>
**Copyright:** 2022 AUTHORS NAMES <br>
**License:** MIT

In [ ]:
# Test for HTML & CSS
# from IPython.display import HTML

# css_str = '<style>.foo{color:white; padding: 10px; font-weight: bold;max-width: 500px; background-color: gray;background-image: linear-gradient(to bottom, rgba(0,0,0,0), rgba(0,0,0,0.25));}</style>'
# html_str = '<div class="foo">Authors:  <br> Copyright: 2022 AUTHORS NAMES <br> License: MIT</div>'

# html = HTML(css_str + html_str)
# display(html)

<div class="alert alert-block alert-success">
<h3>Create a personalized dashboard for requesting data using WEkEO HDA API</h3></div>

# NOTEBOOK INTRODUCTION

### Data used in this example

| Product Description | Link | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-----------------:|:-----------------:|
| ERA5 - Single Levels | <a href="https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview" target="_blank">link</a> | EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3AERA5_HOURLY_VARIABLES_ON_PRESSURE_LEVELS" target="_blank">link</a> |
|CAMS - European Air Quality Forecasts|<a href="https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview" target="_blank">link</a>|EO:ECMWF:DAT:CAMS_EUROPE_AIR_QUALITY_FORECASTS|<a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3ACAMS_EUROPE_AIR_QUALITY_FORECASTS" target="_blank">link</a>|
|CEMS||||
|CMEMS||||

### Learning outcomes

At the end of this notebook you will know;
* How to access WEkEO datasets using HDA API using the request Python library
* How to create a simple dashboard useful to access data directly from your Jupyter Notebook
* How to add interactivity to your scripts using useful widgets


### Outline

Here you could describe your notebook, and perhaps include <a href="https://notebook.wekeo.eu/" target="_blank">links</a> to any relevant external content. 

- Reference: https://www.wekeo.eu/docs/harmonised-data-access-api
- API: https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/ui/#/

Reference used to get the access token follow this page instructions: 
- https://curlconverter.com/ to translate the curl request
- https://www.base64encode.org/ write "username:password" and copy&paste the base64encore in the headers below.

Plotting: 
 - https://mom6-analysiscookbook.readthedocs.io/en/latest/notebooks/Plotting.html


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Login section](#login)   
 2. [Datasets exploration](#exploration) 
 3. [Section 1 - ERA5 Reanalysis Single Levels](#section1)
 4. [Section 2 - CAMS - Europe Air Quality Forecasts](#section2)
 5. [Section 3 - CEMS - EFAS Historical](#section3)
 6. [Section 4 - CMEMS - Analysis/Forecast Sea Bottom Temperature](#section4)


<hr>

<div class="alert alert-info" role="alert">

## <a id='libraries'></a>Libraries

</div>

In [ ]:
import requests
import json
import pandas as pd
import base64
import ipywidgets as widgets
from ipywidgets import Layout
from PIL import Image
import numpy as np
import rioxarray as rxr
import xarray as xr
from IPython.display import display, JSON
import IPython
from urllib.request import urlopen
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, LayersControl
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2
import simplified_WEkEO_methods as m

<div class="alert alert-info" role="alert">

## <a id='login'></a> Login section
[Back to top](#TOC_TOP)

</div>

The following widget allows to login with your WEkEO credentials and obtain the token. Just insert Username and Password without launching the script again (also for other widgets just input the data without launching the code again):

In [ ]:
username = m.text_widget("Insert username: ")
password = m.password_widget("Insert password: ")

login_box = widgets.VBox([username, password])
login_box

In [ ]:
message = str(username.value+":"+password.value).encode('ascii')
base64_message = base64.b64encode(message).decode('ascii')
headers = {'authorization': 'Basic '+base64_message}
token_request = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/gettoken", headers=headers)
token_text = json.loads(token_request.text)
print("Your access token is: "+token_text['access_token'])
headers = {'authorization': token_text['access_token']}
token = list(headers.values())[0]

<div class="alert alert-info" role="alert">

## <a id='exploration'></a> WEkEO Datasets exploration
[Back to top](#TOC_TOP)

</div>

First of all we are going to create a list containing the HDA ID for the chosen datasets used in this notebook. It is possible to access these data using the [*<span style='color:Blue'>requests </span>*](https://docs.python-requests.org/en/latest/) Python library to send the request to the API service. The dataset ID will be used to request the data and it can be retrieved using the HDA API, as follow:

In [ ]:
size = 2000 #number of datasets to be requested
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size="+str(size))
data = json.loads(dataset.text)
data_df = pd.json_normalize(data['content'])
data_df.head()

Another option is to get the dataset ID from the WEkEO portal:

![image_id.jpeg](images/image_id.jpg)

We can add to a list the interesting datasets for our analysis:

In [ ]:
ds_list = ['EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS',
          'EO:ECMWF:DAT:CAMS_EUROPE_AIR_QUALITY_FORECASTS',
          'EO:ECMWF:DAT:EFAS_HISTORICAL',
          'EO:MO:DAT:NWSHELF_ANALYSISFORECAST_PHY_LR_004_001:cmems_mod_nws_phy-bottomt_anfc_7km-2D_P1D-m']

----

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Section 1 - ERA5 Reanalysis Single Levels
[Back to top](#TOC_TOP)

</div>

The first dataset used as example is the ERA5 Reanalysis Single Levels. We already added its HDA ID in the *ds_list* as first step. We can create a dropdown menu to select the dataset we want to use:

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets:")
dataset_id

Always using the request, we can filter the dataset we are interested in:

In [ ]:
size = 2000
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size="+str(size))
data = json.loads(dataset.text)
data_df = pd.json_normalize(data['content'])
data_df = data_df[data_df['datasetId'] == dataset_id.value]
data_df

We now have information about:
 - Abstract
 - HDA dataset ID (we already used it)
 - Dataset preview image
 - Dataset title

For example we can use the dataset information and the metadata to create a simple preview of the dataset:

In [ ]:
# Get the dataset title from data_df
title = data_df.title.values[0]

# Get the description from data_df
description = list(data_df["abstract"])[0]

# Get the dataset image preview and create a display
img_url = list(data_df["previewImage"])[0]
image = IPython.display.Image(img_url, width = 500)
image = widgets.Image(value=image.data,format="jpg", width=500,height=600)

In [ ]:
# Create the boxes
title_box = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr_box = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([title_box, descr_box])

# Create the Layout for the dataset preview (FIX THE LAYOUT)
ui = widgets.AppLayout(header=descr_box, center = image_box, layout=widgets.Layout(border='solid'))
container = widgets.Box([ui])
display(container)

Moreover, is possible to access the metadata for this dataset and prepare the dashboard:

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"+dataset_id.value, headers=headers)
metadata = json.loads(dataset.text)

We can interactively visualize the metadata using the display function for IPython library. This is very useful to understand how the metadata JSON file is structured and how it can be exploited to create the requests:

In [ ]:
display(JSON(metadata))

The ERA5 single level dataset contains many variables that can be selected. We need to obtain a list of variables from the metadata:

In [ ]:
category = metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels']
category_list = []
params_list = []
for item in category:
    category_list.append(item['valuesLabels'])

for item in category_list:
    key_list = list(item.keys())
    params_list.append(key_list)

variables_list = [item for sublist in params_list for item in sublist]

Always keeping the metadata as reference, we can select the other variable necessary for the data request:

In [ ]:
format_type_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
product_type_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
year_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
month_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
day_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
time_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())

In [ ]:
params_sel = m.select_multiple(variables_list, "Variable: ")
product_type_sel = m.select_multiple(product_type_list, "Product type:")
year_sel = m.select_multiple(year_list, "Year: ")
month_sel = m.select_multiple(month_list, "Month: ")
day_sel = m.select_multiple(day_list, "Day: ")
time_sel = m.select_multiple(time_list, "Time: ")
format_type_sel = m.select_buttons(format_type_list, "Format: ", "netcdf")

We can now group all the widgets in a single dashboard interface:

In [ ]:
# Create boxes for the dashboard
title_box= widgets.HTML(value='<h2 style="text-align:center;">'+title+'</h2><hr>')
variables_box = widgets.VBox([title_box, params_sel, product_type_sel, year_sel, month_sel, day_sel, time_sel, format_type_sel])
image_box = widgets.VBox([image])

# Create the dashboard 
ui = widgets.AppLayout(
          left_sidebar=variables_box,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

container = widgets.Box([ui])
display(container)

Selecting the parameter in the dashboard above will automatically change the requested data inside the following query. 

In [ ]:
query = {
  "datasetId": dataset_id.value,
  "multiStringSelectValues": [
    {
      "name": "variable",
      "value": list(params_sel.value)
    },
    {
      "name": "year",
      "value": list(year_sel.value)
    },
    {
      "name": "month",
      "value": list(month_sel.value)
    },
    {
      "name": "day",
      "value": list(day_sel.value)
    },
    {
      "name": "time",
      "value":list(time_sel.value)
    },
    {
      "name": "product_type",
      "value": list(product_type_sel.value)
    }
  ],
  "stringChoiceValues": [
    {
      "name": "format",
      "value": format_type_sel.value
    }
  ]
}

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'authorization': 'Basic '+str(token)
}

data = json.dumps(query)
dataset_post = requests.post("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datarequest", headers=headers, data=data)
job_id = json.loads(dataset_post.text)
print(job_id)
jobId = job_id['jobId']
print("The job ID is: "+jobId+". The status is: "+job_id['status'])

Now that we have the Job ID, we can request the data using (RE-RUN THE FOLLOWING CODE IF NOT WORKING):

In [ ]:
headers = {'authorization': 'Basic '+str(token)}
get_url_request = requests.get('https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest/jobs/'+jobId+'/result', headers=headers)
get_url = json.loads(get_url_request.text)
url = get_url['content'][0]['url']
print('The URL for download is: '+ get_url['content'][0]['url'])

It is possible to click on the URL to download it. Otherwise the following step will be to read it into memory or save it in the folder automatically.

In [ ]:
download_list = ["Download NETCDF", "Read NETCDF in memory"] #Choice - Download data or just read
download_sel = m.select_buttons(download_list, "Data download: ", "Read NETCDF in memory")
download_sel

Save files for netcdf only

In [ ]:
ds = m.download_type(download_sel, download_list, get_url)

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
ds_data = ds[var_drop.value]
ds_data

In [ ]:
f = plt.figure(figsize=(15,10))
p = ds_data.isel(time=0).plot(  #change time
subplot_kws=dict(projection=ccrs.PlateCarree(), facecolor="gray"),
transform=ccrs.PlateCarree())
p.axes.set_global()
p.axes.coastlines()
p.axes.gridlines(color='black', alpha=0.5, linestyle='--')
p.axes.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())

# draw parallels/meridiens and write labels
gl = p.axes.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')

# adjust labels to taste
gl.xlabels_top = False
gl.ylabels_right = False
gl.ylocator = mticker.AutoLocator()
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 18, 'color': 'black'}
gl.ylabel_style = {'size': 18, 'color': 'black'}

----

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Section 2 - CAMS - Europe Air Quality Forecasts
[Back to top](#TOC_TOP)

</div>

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

Select 'EO:ECMWF:DAT:CAMS_EUROPE_AIR_QUALITY_FORECASTS' from the Dropdown list:

In [ ]:
size = 2000
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size="+str(size))
data = json.loads(dataset.text)
data_df = pd.json_normalize(data['content'])
data_df = data_df[data_df['datasetId'] == dataset_id.value]
data_df

In [ ]:
# Get the dataset title from data_df
title = data_df.title.values[0]

# Get the description from data_df
description = list(data_df["abstract"])[0]

# Get the dataset image preview and create a display
img_url = list(data_df["previewImage"])[0]
image = IPython.display.Image(img_url, width = 500)
image = widgets.Image(value=image.data,format="jpg", width=500,height=600)

In [ ]:
# Create the boxes
title_box = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr_box = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([title_box, descr_box])

# Create the Layout for the dataset preview (FIX THE LAYOUT)
ui = widgets.AppLayout(header=descr_box, center = image_box, layout=widgets.Layout(border='solid'))
container = widgets.Box([ui])
display(container)

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"+dataset_id.value, headers=headers)
metadata = json.loads(dataset.text)

In [ ]:
display(JSON(metadata))

In [ ]:
format_type_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
params_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
product_type_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
level_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
type_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
hour_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
leadtime_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())

In [ ]:
params_sel = m.select_multiple(params_list, "Variable:")
product_type_sel = m.select_multiple(product_type_list, "Product type:")
level_sel = m.select_multiple(level_list, "Level: ")
type_sel = m.select_multiple(type_list, "Type: ")
hour_sel = m.select_multiple(hour_list, "Hour: ")
leadtime_sel = m.select_multiple(leadtime_list, "Leadtime hour: ")
format_type_sel = m.select_buttons(format_type_list, "Format: ", "netcdf")
start_date_sel = m.get_date_picker("Select start date: ")
end_date_sel = m.get_date_picker("Select start end: ")

In [ ]:
cams_map, dc = m.draw_map(45, 10, 4)

cams_map

In [ ]:
coords = dc.last_draw['geometry']['coordinates'][0]
W = coords[1][0]
E = coords[3][0]
N = coords[1][1]
S = coords[3][1]

In [ ]:
start_date = metadata['parameters']['dateRangeSelects'][0]['details']['start']
print("The start date for this dataset is: "+start_date)

In [ ]:
title_box = widgets.HTML('<h2>CAMS European air quality forecasts </h2>')
variables_box = widgets.VBox([title_box, params_sel, product_type_sel, level_sel, type_sel, hour_sel, leadtime_sel, start_date_sel, end_date_sel, format_type_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(
          left_sidebar=variables_box,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
query = {
      "datasetId": dataset_id.value,
      "boundingBoxValues": [
        {
          "name": "area",
          "bbox": [
            W,
            N,
            E,
            S
          ]
        }
      ],
      "dateRangeSelectValues": [
        {
          "name": "date",
          "start": start_date_sel.value.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
          "end": end_date_sel.value.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        }
      ],
      "multiStringSelectValues": [
        {
          "name": "variable",
          "value": list(params_sel.value)
        },
        {
          "name": "model",
          "value": list(product_type_sel.value)
        },
        {
          "name": "level",
          "value": list(level_sel.value)
        },
        {
          "name": "type",
          "value": list(type_sel.value)
        },
        {
          "name": "time",
          "value": list(hour_sel.value)
        },
        {
          "name": "leadtime_hour",
          "value": list(leadtime_sel.value)
        }
      ],
      "stringChoiceValues": [
        {
          "name": "format",
          "value": format_type_sel.value
        }
      ]
    }
      
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'authorization': 'Basic '+str(token)}

data = json.dumps(query)
dataset_post = requests.post("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datarequest", headers=headers, data=data)
job_id = json.loads(dataset_post.text)
print(job_id)
jobId = job_id['jobId']
print("The job ID is: "+jobId+". The status is: "+job_id['status'])

In [ ]:
headers = {'authorization': 'Basic '+str(token)}
get_url_request = requests.get('https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest/jobs/'+jobId+'/result', headers=headers)
get_url = json.loads(get_url_request.text)
print(get_url)
url = get_url['content'][0]['url']
print('The URL for download is: '+ get_url['content'][0]['url'])

In [ ]:
download_list = ["Download NETCDF", "Read NETCDF in memory"] #Choice - Download data or just read
download_sel = m.select_buttons(download_list, "Data download: ", "Read NETCDF in memory")
download_sel

In [ ]:
ds = m.download_type(download_sel, download_list, get_url)

In [ ]:
ds = ds.sortby('longitude')

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
ds_data = ds[var_drop.value]
ds_data

In [ ]:
f = plt.figure(figsize=(15,10))
p = ds_data.isel(time=0).plot(  #change time
subplot_kws=dict(projection=ccrs.PlateCarree(), facecolor="gray"),
transform=ccrs.PlateCarree())
p.axes.set_global()
p.axes.coastlines()
p.axes.gridlines(color='black', alpha=0.5, linestyle='--')
p.axes.set_extent([W,E,S,N], ccrs.PlateCarree())

# draw parallels/meridiens and write labels
gl = p.axes.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')

# adjust labels to taste
gl.xlabels_top = False
gl.ylabels_right = False
gl.ylocator = mticker.AutoLocator()
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 18, 'color': 'black'}
gl.ylabel_style = {'size': 18, 'color': 'black'}

----

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Section 3 - CEMS - EFAS Historical
[Back to top](#TOC_TOP)

</div>

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

In [ ]:
data_df = m.get_data_from_name_request(2000, dataset_id.value)
dataset = data_df[data_df['datasetId'] == dataset_id.value]

In [ ]:
title = dataset.title.values[0]
description = m.get_description(data_df, dataset_id)
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
title_box = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr_box = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([title_box, descr])


ui = widgets.AppLayout(header=descr_box,
          center = image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
display(JSON(metadata))

In [ ]:
variable_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
soil_level_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
year_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
month_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
day_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
time_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
system_version_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
model_levels_list = list(metadata['parameters']['stringChoices'][1]['details']['valuesLabels'].keys())
formats_list = list(metadata['parameters']['stringChoices'][2]['details']['valuesLabels'].keys())
download_list = ["Download NETCDF"]

In [ ]:
formats = m.select_buttons(formats_list, "Choose format: ", formats_list[1]) 
system_version_sel = m.select_buttons(system_version_list, "System version:", system_version_list[-1])
model_levels_sel = m.select_buttons(model_levels_list, "Hydrological Model:", model_levels_list[-1])
variable_sel = m.select_multiple(variable_list, "Variable:")
year_sel = m.select_multiple(year_list, "Year:")
month_sel = m.select_multiple(month_list, "Month:")
day_sel = m.select_multiple(day_list, "Day:")
time_sel = m.select_multiple(time_list, "Time:")
soil_level_sel = m.select_multiple(soil_level_list, "Soil level:")
download_sel = m.select_buttons(download_list, "Data download: ", "Download NETCDF")

In [ ]:
title_box = widgets.HTML(value='<h2 style="text-align:center;">'+title+'</h2><hr>')
variables = widgets.VBox([title_box, system_version_sel, variable_sel, model_levels_sel, year_sel, month_sel, day_sel, time_sel, soil_level_sel, formats, download_sel])
image_box = widgets.VBox([image])

ui = widgets.AppLayout(
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
# 02 may 2021 00:00 6hour discharge. works

In [ ]:
job = m.api_query_efas_historical(dataset_id, variable_sel, model_levels_sel, system_version_sel, year_sel, month_sel, day_sel, time_sel, soil_level_sel, formats, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url= m.request_data(jobId, token)

In [ ]:
ds = m.download_type(download_sel, download_list, get_url)

In [ ]:
ds

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
ds_data = ds[var_drop.value].squeeze()

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, projection=ccrs.Orthographic(7, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    global_extent=True, frame_height=400, cmap='viridis',
    coastline=True
)

----

<div class="alert alert-info" role="alert">

## <a id='section4'></a>4. Section 4 - CMEMS - Analysis/Forecast Sea Bottom Temperature
[Back to top](#TOC_TOP)

</div>

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

In [ ]:
data_df = m.get_data_from_name_request(2000, dataset_id.value)
dataset = data_df[data_df['datasetId'] == dataset_id.value]

In [ ]:
title = dataset.title.values[0]
description = m.get_description(data_df, dataset_id)
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
title_box = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([title_box, descr])


ui = widgets.AppLayout(header=descr_box,
          center = image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
display(JSON(metadata))

In [ ]:
variable_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
download_list = ["Download NETCDF"]

In [ ]:
start_date_sel = m.get_date_picker("Select start date: ")
end_date_sel = m.get_date_picker("Select end date: ")
download_sel = m.select_buttons(download_list, "Data download: ", "Download NETCDF")

In [ ]:
end = metadata['parameters']['dateRangeSelects'][0]['details']['end']
start = metadata['parameters']['dateRangeSelects'][0]['details']['start']
print("The start date is "+start+". The end date is "+end)

In [ ]:
cams_map, dc = m.draw_map(50, -10, 4)

cams_map

In [ ]:
coords = dc.last_draw['geometry']['coordinates'][0]
W = coords[1][0]
E = coords[3][0]
N = coords[1][1]
S = coords[3][1]

In [ ]:
tit= widgets.HTML(value='<h2 style="text-align:center;">'+title+'</h2>')
variables = widgets.VBox([tit, start_date_sel, end_date_sel, download_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
job = m.api_query_sea_floor_temperature(dataset_id, start_date_sel, end_date_sel, W, S, E, N, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url= m.request_data(jobId, token)

In [ ]:
order = m.data_order(jobId, get_url, token)
order

In [ ]:
headers = {
    'Accept': 'application/x-netcdf'}
response_order = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/dataorder/download/"+order, headers=headers)

file = open("bottomT.nc", "wb")
file.write(response_order.content)
file.close()

In [ ]:
ds = xr.open_dataset('bottomT.nc')
ds

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
dates = ds.time.values.astype('datetime64[s]').tolist()
date_drop = m.get_dropdown(dates, "Variable: ")
date_drop

In [ ]:
ds_data = ds[var_drop.value].sel(time=date_drop.value).squeeze()

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom'],
                title=ds_data.long_name+" ["+ds_data.units+"]"),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')

ds = ds_data.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')

hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')

hv.element.tiles.OSM() * hv_image_basic

---

# 5 - LAND - Corine (can't download data)

In [ ]:
name = m.text_widget("Insert data name: ")
name #corine

In [ ]:
data_df = m.get_data_from_name_request(2000, name.value)
dataset_list = data_df["datasetId"]
dataset_id = m.get_dropdown(dataset_list, "List of datasets: ")
dataset_id

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )
image

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
formats_list = list(metadata['parameters']['stringChoices'][1]['details']['valuesLabels'].keys())
dataset_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())

In [ ]:
formats = m.select_buttons(formats_list, "Choose format: ", formats_list[1]) 
dataset_name = m.get_dropdown(dataset_list, "Choose dataset: ")

In [ ]:
title= widgets.HTML('<h2 style="text-align:center;">Corine Land Cover</h2><hr><h3 style="padding: 10px;">Input variables - Multiple selection</h3>')
title2 = widgets.HTML('<details><h3>'+str(description[0])+'</div><h3></details>')
variables = widgets.VBox([title, dataset_name, formats])
image_box = widgets.VBox([image])

ui = widgets.AppLayout(header=title2,
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
%load_ext autoreload
%autoreload 2
import WEkEO_methods as m

In [ ]:
job = m.api_query_corine(dataset_id, dataset_name, formats, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url = m.request_data(jobId, token)

In [ ]:
get_url

In [ ]:
order = m.data_order(jobId, get_url, token)
order

In [ ]:
# headers = {
#     'Content-Type': 'application/json',
#     'Accept': 'application/json',
#     'authorization': 'Basic '+str(token)}
# dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/dataorder?status=completed", headers=headers)
# dataset

In [ ]:
headers = {
    'Accept': 'application/zip'}
response_order = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/dataorder/download/"+order, headers=headers)


In [ ]:
import pathlib
import zipfile
import io 
downloaded_file = pathlib.Path.cwd() / "output.zip"

zipfile.ZipFile(downloaded_file.write_bytes(response_order.content),mode='w')

In [ ]:
import zipfile, io, os

with open('test.zip', 'wb') as out_file:
    zipfile.ZipFile(io.BytesIO(response_order.content), mode='w')

In [ ]:
zipfile.ZipFile(io.BytesIO(response_order.content), mode='w')

In [ ]:
zip_filename = "test.zip"
with open(zip_filename, 'wb') as zfile:
    zfile.write(response_order.content)

EEA credentials not available

----

## Datasets exploration

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size=2000")
dataset_text = dataset.text
data = json.loads(dataset_text)

Dataframe containing all the datasets available on WEkEO, with an abstract, a preview image and a title:

In [ ]:
# from JSON to Pandas DataFrame
data_df = pd.json_normalize(data['content'])
data_df

Select the dataset id with a dropdown:

In [ ]:
dataset_list = data_df['datasetId']

datasetId_drop = widgets.Dropdown(
    options=list(dataset_list),
    description='Dataset ID:',
    disabled=False)

datasetId_drop

In [ ]:
datasetId_drop.value

Read abstract corresponding to the dataset ID selected:

In [ ]:
abstract = data_df.loc[data_df['datasetId'] == str(datasetId_drop.value)]

In [ ]:
print(list(abstract["abstract"]))

Plot a preview image of the selected dataset:

In [ ]:
img_url = list(abstract["previewImage"])[0]
im = Image.open(requests.get(img_url, stream=True).raw)
plt.imshow(im)

Request dataset metadata:

In [ ]:
dataset = requests.get("https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"+str(datasetId_drop.value), headers=headers)
dataset_text = dataset.text
dataset = json.loads(dataset_text)

In [ ]:
dataset

Select dataset category:

In [ ]:
category = dataset['parameters']['multiStringSelects'][0]['details']['groupedValueLabels']
category_list = []
for item in category:
     category_list.append(item["label"])

category_drop = widgets.Dropdown(
    options=category_list,
    description='Category:',
    disabled=False)

category_drop

In [ ]:
category_list
category_n = list(range(len(category_list)))

In [ ]:
category_n

In [ ]:
category_dict = dict(zip(category_list, category_n))
category_dict

In [ ]:
parameter = category[category_dict[str(category_drop.value)]]
parameter

In [ ]:
params_list = list(parameter['valuesLabels'].keys())

In [ ]:
params_drop = widgets.Dropdown(
    options=params_list,
    description='Category:',
    disabled=False)

params_drop

In [ ]:
metadata['parameters']['multiStringSelects'][1]

In [ ]:
metadata

# TEST

In [ ]:
if query['stringChoiceValues'][0]['value'] == 'netcdf':
    fl = matches.results[0]['url']
    print('Reading NetCDF file into memory')
    # load into memory 
    with urlopen(fl) as f:
        ds = xr.open_dataset(f.read()).mean(dim='time')
elif query['stringChoiceValues'][0]['value'] == 'zip':
    print('Downloading zip file')
    matches.download()

In [ ]:
import glob, os

zip_list = []
os.chdir("./")
for file in glob.glob("*.zip"):
    print(file)
    zip_list.append(file)

In [ ]:
zip_list

In [ ]:
import ipywidgets as widgets

zip_drop = widgets.Dropdown(
    options=zip_list,
    description='Zip folder:',
    disabled=False,
)
zip_drop

In [ ]:
import zipfile
with zipfile.ZipFile(str(zip_drop.value), 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
nc_list = []
os.chdir("./")
for file in glob.glob("*.nc"):
    print(file)
    nc_list.append(file)

In [ ]:
nc_list

In [ ]:
nc_drop = widgets.Dropdown(
    options=nc_list,
    description='NetCDF folder:',
    disabled=False,
)
nc_drop

In [ ]:
ds = xr.open_dataset(nc_drop.value).mean(dim='time')

In [ ]:
hv_dataset = hv.Dataset(ds)
hv_dataset.data

In [ ]:
ds_data = ds['crwc'].squeeze()

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom']),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv_image_basic

In [ ]:
ds = ds.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')
hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv.element.tiles.OSM() * hv_image_basic

In [ ]:
ds_data = ds[var_drop.value].squeeze()
ds_data

In [ ]:
# f = plt.figure(figsize=(10,8))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
# gl= ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
# gl.right_labels=False
# gl.top_labels=False
# ax.add_feature(cfeature.LAND, zorder=1,edgecolor='k')
# ax.set_extent([-180,180, -90, 90],crs=ccrs.PlateCarree())

# f.suptitle(ds_data.long_name, fontsize=28)
# im = ax.pcolor(ds_data['longitude'].data,ds_data['latitude'].data, ds_data, cmap='plasma')
# cbar= f.colorbar(im,ax=ax)
# cbar.set_label(ds_data.units, rotation =270, fontsize=23,labelpad=15)

In [ ]:
# df = m.get_all_data_request(2000)
# dataset_list = df["datasetId"]